In [1]:
import numpy as np
import pandas as pd
import datetime as dt

import json
from geojson import Feature, FeatureCollection, Point
from math import isnan

from sodapy import Socrata

pd.set_option('display.max_columns', None)

In [2]:
client = Socrata("data.lacity.org", None)

In [3]:
def get_results(date, client):
    results = client.get('wjz9-h9np',
                        where=f'issue_date>="{date}" and \
                            latitude!=99999 and \
                            longitude!=99999 and \
                            location!="TEST"',
                        limit=50000)
    results_df = pd.DataFrame.from_records(results)
    return results_df

In [4]:
raw_df = get_results('2020-07-01', client)

In [5]:
def parse_time(time):
    if len(time) == 3:
        return '0' + time[0] + ':' + time[-2:]
    else:
        return time[0:-2] + ':' + time[-2:]

In [6]:
df = raw_df[pd.notna(raw_df.violation_description)].reset_index(drop=True)

# date and time stored as string, could change for 
df['issue_date'] = [datetime[0:10] for datetime in df.issue_date]
df['issue_time'] = [parse_time(time) for time in df.issue_time]

In [7]:
df.head(5)

,ticket_number,issue_date,issue_time,rp_state_plate,plate_expiry_date,make,body_style,color,location,route,agency,violation_code,violation_description,fine_amount,latitude,longitude,agency_desc,color_desc,body_style_desc,meter_id,marked_time
0,4514737231,2020-07-01,12:43,CA,202012,TOYT,PA,SL,1717 WESTERN AVE,00131,51,22507.8A-,DISABLED PARKING/NO DP ID,363,6468052.623864,1837993.121432,51 - DOT - WESTERN,SILVER,PASSENGER CAR,NaN,NaN
1,4515714932,2020-07-01,16:06,CA,202103,ACUR,PA,BK,600 WILSHIRE PL,00400,54,88.13B+,METER EXP.,63,6474486.820731,1845002.309715,54 - DOT - HOLLYWOOD,BLACK,PASSENGER CAR,VW478,NaN
2,4515722595,2020-07-01,23:20,CA,202010,BMW,PA,BL,1426 HARRIDGE DR,203A,51,22502A,18 IN. CURB/2 WAY,63,6434742.245963,1857684.566024,51 - DOT - WESTERN,BLUE,PASSENGER CAR,NaN,NaN
3,4515573952,2020-07-01,23:15,CA,202104,HYUN,PA,GY,5250 CENTURY BLVD,00204,51,22500.1+,PARKED IN FIRE LANE,63,6449133.736966,1802726.363865,51 - DOT - WESTERN,GREY,PASSENGER CAR,NaN,NaN
4,4515896615,2020-07-01,23:14,PA,201707,SUBA,PA,GY,800 ARDMORE AVE S,402A,54,80.56E4+,RED ZONE,93,6470199.040621,1843515.535669,54 - DOT - HOLLYWOOD,GREY,PASSENGER CAR,NaN,NaN


In [8]:
df.to_csv('sample_data.csv')

In [9]:
# save as geojson
features = df.apply(
    lambda row: Feature(geometry=Point((float(row['longitude']), float(row['latitude'])))),
    axis=1).tolist()

properties = df.drop(['latitude', 'longitude'], axis=1).to_dict('records')

In [10]:
def check_null(val):
    # returns True if None or nan
    if type(val) == str:
        return False
    return isnan(val)

In [11]:
cleaned_properties = []
for d in properties:
    cleaned = {k: d[k] for k in d if not check_null(d[k])}
    cleaned_properties.append(cleaned)

In [12]:
for i in range(len(features)):
    features[i]['properties'] = cleaned_properties[i]

In [13]:
feature_collection = FeatureCollection(features=features)

In [14]:
with open('sample_data.geojson', 'w', encoding='utf-8') as f:
    json.dump(feature_collection, f, ensure_ascii=False)

In [15]:
import geopandas as gpd
test = gpd.read_file('sample_data.geojson')

In [16]:
test.head(5)

,ticket_number,issue_date,issue_time,rp_state_plate,plate_expiry_date,make,body_style,color,location,route,agency,violation_code,violation_description,fine_amount,agency_desc,color_desc,body_style_desc,meter_id,marked_time,geometry
0,4514737231,2020-07-01,12:43,CA,202012,TOYT,PA,SL,1717 WESTERN AVE,00131,51,22507.8A-,DISABLED PARKING/NO DP ID,363,51 - DOT - WESTERN,SILVER,PASSENGER CAR,None,None,POINT (1837993.121 6468052.624)
1,4515714932,2020-07-01,16:06,CA,202103,ACUR,PA,BK,600 WILSHIRE PL,00400,54,88.13B+,METER EXP.,63,54 - DOT - HOLLYWOOD,BLACK,PASSENGER CAR,VW478,None,POINT (1845002.310 6474486.821)
2,4515722595,2020-07-01,23:20,CA,202010,BMW,PA,BL,1426 HARRIDGE DR,203A,51,22502A,18 IN. CURB/2 WAY,63,51 - DOT - WESTERN,BLUE,PASSENGER CAR,None,None,POINT (1857684.566 6434742.246)
3,4515573952,2020-07-01,23:15,CA,202104,HYUN,PA,GY,5250 CENTURY BLVD,00204,51,22500.1+,PARKED IN FIRE LANE,63,51 - DOT - WESTERN,GREY,PASSENGER CAR,None,None,POINT (1802726.364 6449133.737)
4,4515896615,2020-07-01,23:14,PA,201707,SUBA,PA,GY,800 ARDMORE AVE S,402A,54,80.56E4+,RED ZONE,93,54 - DOT - HOLLYWOOD,GREY,PASSENGER CAR,None,None,POINT (1843515.536 6470199.041)
